In [1]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup,NavigableString, Comment

In [12]:

def convert_string(input_str):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'
    
    # Search for the pattern in the string
    match = re.search(pattern, input_str)
    if match:
    # Extract and print the date if found
        date_str = match.group(0)
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        date =  date_obj.strftime('%Y-%m-%d')

        return date
    else:
        'No date found'    
#convert time string to datetime object
def convert_time_string(posted_date):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'

    # Search for the pattern in the string and extract the date
    match = re.search(pattern, posted_date)
    if match:
        date_str = match.group(0)
        # Convert the extracted date string to datetime object
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        return date_obj
    else:
        return None 
def get_time_string(date_str):
    if len(date_str)<=5:
        crawl_time = datetime.fromtimestamp(time.time())
        year = str(crawl_time.year)
        date_str = date_str+"/"+year
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        date =  date_obj.strftime('%Y-%m-%d')
    else:
        date = convert_string(date_str)
        date_obj = datetime.strptime(date,'%Y-%m-%d')
    return date_obj, date


def get_content_autodaily(url):
    # def get_content_autodaily()
    response = requests.get(url)
    time.sleep(5)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('h1').text.strip()
    date = soup.find('time').text.strip()
    published_date = convert_string(date)
    article = soup.find('div', class_ = 'article-detail')
    for script_or_style in article(['script', 'style','iframe']):
                script_or_style.decompose()
    related_item = article.find('div', class_ = 'item-relate')
    related_item.decompose()
    author_item = article.find_all(recursive = True)[-2:]
    for i in author_item:
        i.decompose()
    caption_text_list = article.find_all('em')
    tags_to_remove = article.find_all(['a', 'span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
        # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip()
    #for i in article.find_all('img'):
        #i.attrs = ['class', 'alt', 'src', 'data-original']
        #del i['onclick']
        #del i['style']
        #del i['class']
        #del i['alt']
    #remove all image attributes except somes from list
    list_attr = ['src','alt','data-src']
    for i in article.find_all('img'):
        for j in list(i.attrs.keys()):
            if j not in list_attr:
                i.attrs.pop(j)
    img_list = article.find_all('img')
    n_img = len(img_list)
    #print(len(caption_text_list))
    for i in range(0,n_img):

        caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
        try:
            caption_text = NavigableString(caption_text_list[i].get_text())
            caption_end = NavigableString("[/caption]")
            # Insert the custom tags and caption text around the <img> tag
            img_list[i].insert_before(caption_start)
            img_list[i].insert_after(caption_end)
            img_list[i].insert_after(caption_text) 
        except IndexError as e:
            print(e)
        
    for i in article.find_all('img'):
        i['src'] = i['data-src']
    for i in caption_text_list:
        i.decompose()
        #print(img_list[i]['src'])
        #caption_text_list[i].decompose()
    for i in article.find_all(recursive = True):
        try:
            del i['onclick']
            i['id'] =''
            i['class']=''
            i['style'] =''
            i['href'] = ''
        except AttributeError:
            continue
        #try:
            #print(i)
            
        except TypeError:
            continue
    for i in article.find_all('img'):
        i['class'] = "aligncenter"
        i['width'] = 800
        i['height'] = 400
        del i['data-src']
    for item in article.find_all('div'):
        if item.string =="":
            item.decompose()
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: autodaily.vn"  # Set the content of <i> tag
    
    # Append the <i> tag as the last child of the <article> tag
    article.append(source_tag)
    #Highlight the fist paragraph by using <strong> tag
    p_tag_1st = article.find_all('p')[0]
    p_text = p_tag_1st.get_text()
    # Create a new <strong> tag with the same text
    strong_tag = soup.new_tag("strong")
    strong_tag.string = p_text

    # Replace the <p> tag with the <strong> tag
    p_tag_1st.replace_with(strong_tag)
    return article, title, published_date
def get_post(url):
    try:
        content,title,published_date = get_content_autodaily(url)
        return content,title,published_date
    except AttributeError as e:
        print(e)
def get_list_url(cate_url):
    response = requests.get(cate_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    list = soup.find_all('article', class_ = 'eva-news-trend-h-items d-flex')
    urls = []
    for i in list:
        url = i.find('a')['href']
        urls.append(url)
    return urls
def filter_list(urls):
    filtered_urls = []
    crawl_time = datetime.fromtimestamp(time.time()-3*24*3600)
    for i in urls:
        response = requests.get(i)
        time.sleep(2)
        soup = BeautifulSoup(response.content, 'html.parser')
        try:
            #wrapper = soup.find('div', class_ = "col780 left clearafter")
            #date_posted = wrapper.find('p',class_ ='news-time left').text.strip()
            #date_posted_norm = convert_time_string(date_posted)
            date_str = soup.find('time').text.strip()
            #print(date_str)
            date_posted_norm = get_time_string(date_str)[0]
            if ( (date_posted_norm.day == crawl_time.day) and (date_posted_norm.month == crawl_time.month) and (date_posted_norm.year == crawl_time.year) ):
                filtered_urls.append(i)
                #print(i)
        except AttributeError as e:
            print(e)
            continue
    return filtered_urls
#add list url to json
#add list url to json
def add_list(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):  
            urls = get_list_url(web_json_obj['urls'][i]['sub-category'][j]['url'])
            print(i,j,web_json_obj['urls'][i]['sub-category'][j]['url'])
            web_json_obj['urls'][i]['sub-category'][j]['url_list'] = filter_list(urls)
# add post content from get content function to json object
def add_post(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):
            web_json_obj['urls'][i]['sub-category'][j]['content'] = {}
            list_key = [v for v in range(0,len(web_json_obj['urls'][i]['sub-category'][j]['url_list']))]
            for u in list_key:
                web_json_obj['urls'][i]['sub-category'][j]['content'][u] = {}
                if u != "":
                    web_json_obj['urls'][i]['sub-category'][j]['content'][u]['text'] ,web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['published_date'] = get_post(web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
                    print(i,j,web_json_obj['urls'][i]['sub-category'][j]['cate_id'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
#add all necessary information to json object
def get_news_autodaily():
    _autodaily= {
            "home_page":"https://autodaily.vn/",
            "urls":{
                "Nhân vật đẹp":
                {
                 "url":"https://eva.vn/nhan-vat-dep-c262.html#",
                 "sub-category":{  
                    0:{"name":"Người đẹp",
                     "url":"https://eva.vn/nhan-vat-dep-c262.html",
                     "cate_id":63,
                      "url_list" : []},
                 }
                }
            }
        }
#
    add_list(_autodaily)
    add_post(_autodaily)
    return _autodaily
#send post content to wordpress via endpoint
def send_post_to_5goals(title,content,category_id,published_date):
    # URL of the API endpoint (this is a placeholder and needs to be replaced with the actual URL)
    url = "https://api2023.5goal.com/wp-json/custom/createPost"
    
    # Data to be sent in the POST request
    data = {
        "title": title,
        "content": content,
        "category_id": category_id,
        "token": 'draftpost',#'5goalvodichcmnl',  # Replace with your actual access token
        "published_date": published_date,
        "domain":"autodaily"
          # Replace with the actual category ID as required
    }
    
    # Sending the POST request
    response = requests.post(url, data=data)
    
    # Checking the response
    if response.status_code == 200:
        print("The post was successfully created.")
        print("Response:", response.text)  # Prints the response text from the server
    else:
        print(f"Failed to create the post. Status code: {response.status_code}")
def main():
    _autodaily = get_news_autodaily()
    for i in list(_autodaily['urls'].keys()):
    #web_24h_com_vn2['url'][i]['cate_id']
        for j in list(_autodaily['urls'][i]['sub-category']):
            url_list =  _autodaily['urls'][i]['sub-category'][j]['url_list']
            print(url_list)
            for t in range(0,len(url_list)):
                content = _autodaily['urls'][i]['sub-category'][j]['content'][t]['text']
                title = _autodaily['urls'][i]['sub-category'][j]['content'][t]['title']
                published_date = _autodaily['urls'][i]['sub-category'][j]['content'][t]['published_date']
                cate_id = _autodaily['urls'][i]['sub-category'][j]['cate_id']
                print(title, url_list[t])
                #send_post_to_5goals(title,str(content), cate_id, published_date)
                try:
                    text_len = len(text.text)
                    if text_len <450:
                        print(text.text)
                        continue
                    else:
                         send_post_to_5goals(title,str(text),cate_id,published_date)
                except (AttributeError,TypeError):
                    continue


In [8]:
def get_list_url(cate_url):
    response = requests.get(cate_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    list = soup.find_all('article', class_ = 'eva-news-trend-h-items d-flex')
    urls = []
    for i in list:
        url = i.find('a')['href']
        urls.append(url)
    return urls

In [9]:
get_list_url('https://eva.vn/nhan-vat-dep-c262.html')

['https://eva.vn/nhan-vat-dep/bat-gap-vo-giam-doc-cua-luong-xuan-truong-da-trang-sang-visual-uy-tin-danh-bai-cam-thuong-c262a579195.html',
 'https://eva.vn/nhan-vat-dep/lo-lem-nha-quyen-linh-dien-ao-nhat-binh-hoa-my-nhan-xua-anh-chup-len-lo-nhan-sac-gay-ngo-ngang-c262a579183.html',
 'https://eva.vn/nhan-vat-dep/ngo-ngang-voi-nhan-sac-cua-co-gai-nuong-khoai-ven-duong-nguoi-qua-lai-deu-phai-ngoai-nhin-c262a579168.html',
 'https://eva.vn/nhan-vat-dep/hoa-hau-ngan-nam-co-mot-khoe-anh-the-dep-nhu-mo-van-thua-xa-mot-dan-chi-kin-tieng-c262a579112.html',
 'https://eva.vn/nhan-vat-dep/truy-tim-co-gai-co-guong-mat-ty-le-kim-cuong-so-do-3-vong-hoan-hao-khong-cho-che-c262a579060.html',
 'https://eva.vn/nhan-vat-dep/con-gai-cua-marian-rivera-len-8-tuoi-co-rang-tho-kem-tham-my-van-xinh-dep-chuan-cong-chua-doi-thuc-c262a579033.html',
 'https://eva.vn/nhan-vat-dep/xuat-hien-nang-tho-moi-cua-man-anh-viet-dep-trong-veo-khong-can-son-phan-c262a578975.html',
 'https://eva.vn/nhan-vat-dep/mai-phuong-thuy-k

In [ ]:
def get_content_autodaily(url):
    # def get_content_autodaily()
    response = requests.get(url)
    time.sleep(5)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('h1', id = 'tieu_de_bai_viet').text.strip()
    date = soup.find('div', class_ = 'eva-author-time-art mar-b-25').text.strip()
    published_date = convert_string(date)
    article = soup.find('div', id = 'centerContent')
    for script_or_style in article(['script', 'style','iframe']):
                script_or_style.decompose()
    related_item = article.find('div', class_ = 'item-relate')
    related_item.decompose()
    author_item = article.find_all(recursive = True)[-2:]
    for i in author_item:
        i.decompose()
    caption_text_list = article.find_all('em')
    tags_to_remove = article.find_all(['a', 'span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
        # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip()
    #for i in article.find_all('img'):
        #i.attrs = ['class', 'alt', 'src', 'data-original']
        #del i['onclick']
        #del i['style']
        #del i['class']
        #del i['alt']
    #remove all image attributes except somes from list
    list_attr = ['src','alt','data-src']
    for i in article.find_all('img'):
        for j in list(i.attrs.keys()):
            if j not in list_attr:
                i.attrs.pop(j)
    img_list = article.find_all('img')
    n_img = len(img_list)
    #print(len(caption_text_list))
    for i in range(0,n_img):

        caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
        try:
            caption_text = NavigableString(caption_text_list[i].get_text())
            caption_end = NavigableString("[/caption]")
            # Insert the custom tags and caption text around the <img> tag
            img_list[i].insert_before(caption_start)
            img_list[i].insert_after(caption_end)
            img_list[i].insert_after(caption_text) 
        except IndexError as e:
            print(e)
        
    for i in article.find_all('img'):
        i['src'] = i['data-src']
    for i in caption_text_list:
        i.decompose()
        #print(img_list[i]['src'])
        #caption_text_list[i].decompose()
    for i in article.find_all(recursive = True):
        try:
            del i['onclick']
            i['id'] =''
            i['class']=''
            i['style'] =''
            i['href'] = ''
        except AttributeError:
            continue
        #try:
            #print(i)
            
        except TypeError:
            continue
    for i in article.find_all('img'):
        i['class'] = "aligncenter"
        i['width'] = 800
        i['height'] = 400
        del i['data-src']
    for item in article.find_all('div'):
        if item.string =="":
            item.decompose()
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: autodaily.vn"  # Set the content of <i> tag
    
    # Append the <i> tag as the last child of the <article> tag
    article.append(source_tag)
    #Highlight the fist paragraph by using <strong> tag
    p_tag_1st = article.find_all('p')[0]
    p_text = p_tag_1st.get_text()
    # Create a new <strong> tag with the same text
    strong_tag = soup.new_tag("strong")
    strong_tag.string = p_text

    # Replace the <p> tag with the <strong> tag
    p_tag_1st.replace_with(strong_tag)
    return article, title, published_date

In [95]:
response = requests.get('https://eva.vn/nhan-vat-dep/con-gai-cua-marian-rivera-len-8-tuoi-co-rang-tho-kem-tham-my-van-xinh-dep-chuan-cong-chua-doi-thuc-c262a579033.html')
soup = BeautifulSoup(response.content, 'html.parser')
title = soup.find('h1', id = 'tieu_de_bai_viet').text.strip()
date = soup.find('div', class_ = 'eva-author-time-art mar-b-25').text.strip()
published_date = convert_string(date)
article = soup.find('article', class_ = 'eva-cont-art no-copy')
for script_or_style in article(['script', 'style','iframe']):
    script_or_style.decompose()
videos = article.find_all('div', class_ = ' v-24hContainer_zplayer_90d8e1_loaded')
for video in videos:
    video.decompose()


In [96]:
#
related_article =  article.find('div', class_ = 'evtBox evtBoxPrBt')
related_article.decompose()

In [97]:
article

<article class="eva-cont-art no-copy">
<div class="eva-cont-art__sum">
<div class="nsSap sap clrF">
<h2 class="fw-semi-bold tuht_show">Nhan sắc của Zia càng lớn càng khiến netizen phải trầm trồ. Nhóc tì như thể phiên bản "tiểu" Marian Rivera song xét khoản tài năng thì cô bé "ăn đứt" mẹ.</h2>
</div>
</div>
<!-- Bai Audio -->
<section class="banr-hot mrT20 mrB20" id="bnBAER_119_15s_container"><div class="banrpstn">
<div class="txtbanrHrzt txtrt" id="bnbaer_by_bnBAER_119_15s"><span class="displaynone"></span></div>
<div class="" id="bnBAER_119_15s">

</div>
</div></section> <!-- End Bai Audio -->
<div class="eva-cont-art__info" id="baiviet-container">
<p>Nói không quá khi cho rằng gia đình Marian Rivera và Dingdong Dantes là tổ ấm quyền lực bậc nhất châu Á. Chỉ tính riêng trang fanpage của Marian Rivera đã có hơn 29 triệu người theo dõi, trang instagram có 12,7 triệu người bấm follow. Gia đình quyền lực của xứ sở biển đảo còn nổi tiếng rần rần trong hội bỉm sữa khi có 2 con: 1 nếp và 1 t

In [98]:
for i in article.find_all('img'):
    i['src'] = i['data-original']
   

In [99]:
list_captions = []
for p_tag in article.find_all('p', class_ = 'img_chu_thich_0407 eva-cont-art__cap-img text-center'):
    if p_tag.text != None:
        list_captions.append(p_tag.text.strip())


In [100]:
list_captions

['Nhóc tì mới đây vừa được tổ chức bữa tiệc sinh nhật 8 tuổi linh đình. Em bé khoe sắc vóc như công chúa trong truyện cổ tích với chiếc đầm xòe xinh xắn.',
 'Con gái gia đình giàu có bậc nhất Philippines lộng lẫy như truyện cổ tích Bạch Tuyết và 7 chú lùn.',
 'Em bé được trang điểm nhẹ nhàng và làm tóc xoăn xõa, đầu đội vương miện.',
 'Tấm hình chụp này còn khiến nhiều fans trầm trồ vì quá giống Marian Rivera. Nhiều comment ví von Zia là tiểu Marian Rivera.',
 'Zia là tinh túy của cả bố lẫn mẹ, song em bé có lẽ giống mẹ hơn cả. Tấm hình chụp ngày bé của Marian Rivera khiến nhiều người bối rối vì cứ ngỡ ảnh của Zia đã được chỉnh màu.',
 'Càng lớn, Zia càng xinh đẹp và được dự đoán sẽ "tiếp quản" danh hiệu "Mỹ nhân đẹp nhất Philippines" của mẹ trong tương lai. Không chỉ xinh đẹp, dưới sự nuôi dạy của mẹ, Zia còn thông minh, học giỏi.',
 'Zia càng lớn càng hoàn thiện ngũ quan trên gương mặt. Nhóc tì mang vẻ lém lỉnh với đôi mắt cười, khuôn miệng tươi tắn và đặc biệt là chiếc răng thỏ.',
 

In [77]:
for i in article.find_all('img'):
    parent_element = i.parent
    next_sibling_of_parent = parent_element.find_next_sibling('p')
    try:
        print(next_sibling_of_parent.text.strip(), "______________")
    except AttributeError:
        continue
    

Em bé được trang điểm nhẹ nhàng và làm tóc xoăn xõa, đầu đội vương miện. ______________
Tấm hình chụp này còn khiến nhiều fans trầm trồ vì quá giống Marian Rivera. Nhiều comment ví von Zia là tiểu Marian Rivera. ______________
Zia là tinh túy của cả bố lẫn mẹ, song em bé có lẽ giống mẹ hơn cả. Tấm hình chụp ngày bé của Marian Rivera khiến nhiều người bối rối vì cứ ngỡ ảnh của Zia đã được chỉnh màu. ______________
Càng lớn, Zia càng xinh đẹp và được dự đoán sẽ "tiếp quản" danh hiệu "Mỹ nhân đẹp nhất Philippines" của mẹ trong tương lai. Không chỉ xinh đẹp, dưới sự nuôi dạy của mẹ, Zia còn thông minh, học giỏi. ______________
Zia càng lớn càng hoàn thiện ngũ quan trên gương mặt. Nhóc tì mang vẻ lém lỉnh với đôi mắt cười, khuôn miệng tươi tắn và đặc biệt là chiếc răng thỏ. ______________
Cặp mẹ con còn từng chụp hình cùng nhau với tư thế show cặp răng cửa như răng thỏ. Trông cả hai xinh xắn và đáng yêu như 2 giọt nước. ______________
Zia đang ở độ tuổi thay răng nên lại càng trông đáng yêu

In [81]:
html_content = '''
<html>
  <body>
    <div class="header">Header content</div>
    <div class="main">Main content</div>
    <div class="footer">Footer content</div>
  </body>
</html>
'''
soup = BeautifulSoup(html_content, 'html.parser')
current_element = soup.find(class_='footer')
previous_sibling = current_element.find_previous_sibling()
print(previous_sibling)
# To find the previous sibling with a specific class:
# previous_sibling = current_element.find_previous_sibling(class_='specific-class')


<div class="main">Main content</div>
